In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Parameters
n = 10  # Length of each vector
num_samples = 10000  # Number of samples for training
num_test_samples = 1000  # Number of samples for testing

# Data Generation Function
def generate_data(num_samples, vector_length):
    vectors_a = np.random.rand(num_samples, vector_length)
    vectors_b = np.random.rand(num_samples, vector_length)
    concatenated_vectors = np.concatenate((vectors_a, vectors_b), axis=1)
    dot_products = np.einsum('ij,ij->i', vectors_a, vectors_b)
    return torch.tensor(concatenated_vectors, dtype=torch.float32), torch.tensor(dot_products, dtype=torch.float32)

# Neural Network Architecture
class DotProductNN(nn.Module):
    def __init__(self):
        super(DotProductNN, self).__init__()
        self.fc1 = nn.Linear(2 * n, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Instantiate model, loss function, and optimizer
model = DotProductNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Data
X_train, y_train = generate_data(num_samples, n)

# Training Loop
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    loss.backward()
    optimizer.step()

# Generate Test Data
X_test, y_test = generate_data(num_test_samples, n)

# Testing
with torch.no_grad():
    outputs_test = model(X_test)
    test_loss = criterion(outputs_test, y_test.view(-1, 1))

test_loss.item()


6.724447727203369

In [52]:
# Re-instantiating the model and optimizer for a fresh start
model = DotProductNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Generate Test Data (same as before)
X_test, y_test = generate_data(num_test_samples, n)

# Testing before training
with torch.no_grad():
    outputs_test_before = model(X_test)
    test_loss_before = criterion(outputs_test_before, y_test.view(-1, 1))

# Training Loop
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    loss.backward()
    optimizer.step()

# Testing after training
with torch.no_grad():
    outputs_test_after = model(X_test)
    test_loss_after = criterion(outputs_test_after, y_test.view(-1, 1))

test_loss_before.item(), test_loss_after.item()


(6.791478157043457, 0.21909284591674805)

In [60]:
single_vector = torch.randn(20).unsqueeze(0)
print("expected ", torch.dot(single_vector.squeeze()[:10], single_vector.squeeze()[10:]))
print("actual ", model(single_vector))

expected  tensor(-2.9705)
actual  tensor([[1.4193]], grad_fn=<AddmmBackward0>)
